In [0]:
from google.colab import files
import pandas as pd

In [0]:
uploaded = files.upload()

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [20]:
!kaggle datasets download yongho1037/smsspamcollection

  0% 0.00/206k [00:00<?, ?B/s]
100% 206k/206k [00:00<00:00, 77.4MB/s]


In [0]:
mkdir dataset

In [22]:
! unzip smsspamcollection.zip -d dataset

Archive:  smsspamcollection.zip
  inflating: dataset/SMSSpamCollection  


In [23]:
ls

dataset/  kaggle.json  sample_data/  smsspamcollection.zip


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import sklearn
import sys

In [25]:
df=pd.read_table("/content/dataset/SMSSpamCollection",header = None, encoding="utf-8")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
0    5572 non-null object
1    5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [26]:
classes = df[0]
classes.value_counts()

ham     4825
spam     747
Name: 0, dtype: int64

In [27]:
from sklearn.preprocessing import LabelEncoder
enco = LabelEncoder()
Y = enco.fit_transform(classes)
Y[:10]

array([0, 0, 1, 0, 0, 1, 0, 0, 1, 1])

In [28]:
msg = df[1]
msg[:10]

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object

In [0]:
#replace email address with emailaddr
processed = msg.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddr')

#replace urls with webaddrs
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddrs')

#replace money symbols with moneysymb
processed = processed.str.replace(r'\$','moneysymb')

#replace 10 digits phone no with phone nos
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$','phonenos')

#replace normal nos with number
processed = processed.str.replace(r'\d+(\.\d+)?','number')

In [0]:
#remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

#replace whitespaces between form with one single space

processed = processed.str.replace(r'\s+', ' ')

#remove leading and trailing whitespaces

processed = processed.str.replace(r'^\s+|\s+?$','')

In [31]:
#change words with lower case
processed = processed.str.lower()
processed

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in number a wkly comp to win fa cup...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
5       freemsg hey there darling it s been number wee...
6       even my brother is not like to speak with me t...
7       as per your request melle melle oru minnaminun...
8       winner as a valued network customer you have b...
9       had your mobile number months or more u r enti...
10      i m gonna be home soon and i don t want to tal...
11      six chances to win cash from number to number ...
12      urgent you have won a number week free members...
13      i ve been searching for the right words to tha...
14                      i have a date on sunday with will
15      xxxmobilemovieclub to use your credit click th...
16                                 oh k i m watching here
17      eh u r

In [32]:
!pip install nltk

In [33]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [35]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))

In [0]:
from nltk.tokenize import word_tokenize

#creating a bag of words
all_words=[]

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

In [38]:
#print total no of words and 15 most common words
print("total no of words{}".format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

total no of words6562
Most common words: [('number', 3071), ('u', 1207), ('call', 679), ('go', 456), ('get', 451), ('ur', 391), ('gt', 318), ('lt', 316), ('come', 304), ('ok', 293), ('free', 284), ('day', 276), ('know', 275), ('love', 266), ('like', 261)]


In [0]:
word_features= list(all_words.keys())[:]

In [40]:
#define a find features function
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word]=(word in words)
        
    return features

#example
feature = find_features(processed[0])
for key, value in feature.items():
    if value ==True:
        print (key)
   

go
jurong
point
crazi
avail
bugi
n
great
world
la
e
buffet
cine
got
amor
wat


In [0]:
#find features for all messages
messages = list(zip(processed, Y))

#seed for reproducibility
seed =1
np.random.seed =seed
np.random.shuffle(messages)

#call find_features function for each SMS msg
featuresets = [(find_features(text),label) for (text,label) in messages]

In [0]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(featuresets,test_size=0.25,random_state=seed)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [0]:
#define models to train
names=['K Nearest Neighbors','Decision Tree','Random Forest','Logistic Regression','SGD Classifier','Naive Bayes','SVM Linear']

classifier=[
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter=100),
    MultinomialNB(),
    SVC(kernel='linear')
]

models = zip(names, classifier)

models = list(models)
#models

In [45]:
#wrap models in nltk
from nltk.classify.scikitlearn import SklearnClassifier

for name,model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(train)
    accuracy = nltk.classify.accuracy(nltk_model,test) * 100
    print('{} Accuracy: {}'.format(name, accuracy))

K Nearest Neighbors Accuracy: 92.67767408470927
Decision Tree Accuracy: 96.91313711414213


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest Accuracy: 96.33883704235463


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy: 98.20531227566404
SGD Classifier Accuracy: 98.1335247666906
Naive Bayes Accuracy: 97.05671213208902
SVM Linear Accuracy: 97.98994974874373


In [46]:
#emsemble method - voting classifier
from sklearn.ensemble import VotingClassifier

#define models to train
names=['K Nearest Neighbors','Decision Tree','Random Forest','Logistic Regression','SGD Classifier','Naive Bayes','SVM Linear']

classifier=[
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter=100),
    MultinomialNB(),
    SVC(kernel='linear')
]

models = list(zip(names, classifier))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators=models , voting ='hard', n_jobs=-1))
nltk_ensemble.train(train)
accuracy = nltk.classify.accuracy(nltk_ensemble,test) * 100
accuracy

98.27709978463747

In [0]:
#class label prediction
text_features , labels = zip(*test)

prediction = nltk_ensemble.classify_many(text_features)

In [49]:
#print confusion matrix and classification report
print(classification_report(labels,prediction))

pd.DataFrame(confusion_matrix(labels, prediction),
            index = [['actual', 'actual'],['ham','spam']],
            columns = [['predicted','predicted'],['ham','spam']])

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1193
           1       0.99      0.89      0.94       200

    accuracy                           0.98      1393
   macro avg       0.99      0.94      0.96      1393
weighted avg       0.98      0.98      0.98      1393



predicted     
                  ham spam
actual ham       1192    1
       spam        23  177